In [1]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from katonic.ml import MyClient

In [2]:
myclient = MyClient()
mlflow = myclient.mlflow
client = myclient.client

In [3]:
def metric(actual, pred):
    acc_score = accuracy_score(actual, pred)
    recall = recall_score(actual, pred, average='weighted')
    precision_scr = precision_score(actual, pred, average='weighted')
    f1_scr = f1_score(actual, pred, average='weighted')
    
    return (
        acc_score,
        recall,
        f1_scr,
        precision_scr
    )

In [10]:
exp_name = "mlflow-test-catboost"
mlflow.set_experiment(exp_name)
exp_details = mlflow.get_experiment_by_name(exp_name)

data = load_iris()
feature_names = [
    name.replace(" ", "_").replace("(", "").replace(")", "")
    for name in data["feature_names"]
]
X_train, X_test, y_train, y_test = train_test_split(
    data["data"], data["target"], test_size=0.2
)

with mlflow.start_run(run_name=exp_name):
    kwargs = {}
    kwargs['iterations']=100
    kwargs['depth']=6
    kwargs['learning_rate']=0.1
    kwargs['loss_function']='MultiClass',
    kwargs['verbose']=False

    model = CatBoostClassifier(
        iterations=100, depth=6, learning_rate=0.1,
        loss_function='MultiClass', verbose=False)
    model.fit(X_train, y_train)
    mlflow.log_param("iterations", 100)

    y_pred = model.predict(X_test)

    (acc_score, recall, f1_scr, precision_scr) = metric(y_test, y_pred)

    model_metrics = {
        "accuracy_score": acc_score,
        "recall": recall,
        "f1_score": f1_scr,
        "precision_score": precision_scr
    }
    for k, v in kwargs.items():
        mlflow.log_param(k, v)
    for metric_name, score in model_metrics.items():
        mlflow.log_metric(metric_name, score)

    model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="model")
    